In [1]:
import requests
import time
import hashlib

In [2]:
YOUDAO_URL = "https://dict.youdao.com/jsonapi_s"

YOUDAO_QUERY = {
    "doctype": "json",
    "jsonversion": 4
}

APP_KEY = "Mk6hqtUp33DGGtoS63tTJbMUYjRrG1Lu"


def YOUDAO_WORD(word):
    return f"q={word}&appKey=1c81b216d114ced1&salt=1660396042212&from=en&to=zh-CHS&sign=e5dab2a7bd31e67f3959ed79e6db2083082d6a74166c46324a106d7099f4b600&signType=v3&curtime={int(time.time())}&headers%5BContent-Type%5D=application%2Fx-www-form-urlencoded"


from enum import Enum


def encrpted(data: str):
    return hashlib.md5(data.encode('utf-8')).hexdigest()


# le:语言类型
def api_data(word: str, dicts="dicts=ce&dicts=ec", language: str = "en", client="web", keyfrom="webdict"):
    time = len(word + keyfrom) % 10
    salt = encrpted(word + keyfrom)
    sign = encrpted(client + word + str(time) + APP_KEY + salt)
    # print(client + word + str(time) + APP_KEY + salt)
    data = f"q={word}&le={language}&t={time}&client=web&sign={sign}&keyfrom={keyfrom}"
    return data


class DicSource(Enum):
    UNCLEAR = "unclear"
    YOUDAO = "youdao"



In [3]:

class Word:
    word: str = "",
    usphonetic: str = "",
    ukphonetic: str = "",
    meaning: str = "",
    example: str = "",
    source: DicSource = DicSource.UNCLEAR,
    exam: str = "",

    def __init__(self, word: str, usphonetic: str,
                 ukphonetic: str, meaning: str, example: str,
                 source: DicSource, exam: str):
        self.word = word
        self.usphonetic = usphonetic
        self.ukphonetic = ukphonetic
        self.meaning = meaning
        self.example = example
        self.source = source
        self.exam = exam

    def __str__(self):
        return "word : %s,usphonetic : %s, ukphonetic : %10s, meaning : %s, example : %s, exam: %s, source : %s" % (
            self.word, self.usphonetic, self.ukphonetic, self.meaning, self.example, self.exam, self.source)




In [5]:
import logging


# obtain all information from youdao's api.
def __parse_word_from_youdao(word: str):
    try:
        details = requests.post(YOUDAO_URL, params=YOUDAO_QUERY, data=api_data(word), headers={
            "content-type": "application/x-www-form-urlencoded"
        }).json()
        # concise dictionary
        if "ec" in details and "word" in details.get("ec", ""):
            basic = details['ec']
            trs = basic['word']['trs']
            # print(trs)
            meaning = ""
            for tr in trs:
                meaning += tr.pop("pos", "") + tr['tran'] + ';'
            return Word(
                basic['word']['return-phrase'],
                usphonetic=basic['word'].pop("usphone", ""),
                ukphonetic=basic['word'].pop("ukphone", ""),
                meaning=meaning,
                example=details['blng_sents_part']['sentence-pair'][0]['sentence'],
                source=DicSource.YOUDAO,
                exam="，".join(basic.pop('exam_type', ["未知"]))
            )
        else:
            return None

    except ValueError:
        print("Parsing value failed!", word)
    except NameError:
        print("Parsing name failed!", word)
    except KeyError as error:
        print("Parsing key failed!", word)
        logging.exception(error)


def query_word(source: DicSource, word: str):
    if source == DicSource.YOUDAO:
        return __parse_word_from_youdao(word)

In [6]:
print(api_data("intricate"))

q=intricate&le=en&t=6&client=web&sign=7cd29ed159942a0107a21a80fe634581&keyfrom=webdict


In [7]:
demo_word = query_word(DicSource.YOUDAO, "rent")
print(demo_word)

Parsing value failed! rent
None


In [8]:
import csv

csv_reader = csv.reader(open("./形近单词.txt"))
word_list = []
for line in csv_reader:
    for word in line:
        if word.strip() != '':
            word_list.append(word.strip())

FileNotFoundError: [Errno 2] No such file or directory: './形近单词.txt'

In [ ]:
# 创建一个csv文件，存放生成的单词
headers = ["单词", "美音", "英音", "考试等级", "意思", "例句", "来源"]
start = 0
failed = []
with open("生成的形近单词.csv", "w+", encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    # 先写入列名
    if start == 0:
        csv_writer.writerow(headers)
    counter = 0
    cw = ""
    for word in word_list:
        try:
            cw = word
            if start <= counter:
                current_word = query_word(DicSource.YOUDAO, word)
                if current_word is not None:
                    csv_writer.writerow([current_word.word, current_word.usphonetic, current_word.usphonetic,
                                         current_word.exam, current_word.meaning,
                                         current_word.example, current_word.source.value])
                else:
                    print("查询失败:", word, counter)
            counter += 1
        except TypeError as error:
            print("TypeError", cw, ";counter:", counter)

In [ ]:
print(time.time())

1660465429.1471472
1660465463.2886968
